In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

sentences = ["This is an example sentence", "Each sentence is converted"]

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

print(f"{encoded_input=}")
with torch.no_grad():
    model_output = model(**encoded_input)
    print(f"{model_output['last_hidden_state'].shape=}")
    print(f"{model_output['pooler_output'].shape=}")
# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
print(f"{sentence_embeddings.shape=}")
print("Sentence embeddings:")
print(sentence_embeddings)